In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import re
from math import floor
from random import randint

In [2]:
import os
os.path.exists('train_xml/')

True

In [3]:
train_labels = glob('train_xml/*.xml')
drug_label_text = []
for label in tqdm(train_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    for mention in soup.find_all('Mention'):
        if mention['type'] == "AdverseReaction":
            section = mention['section']
            start = mention['start']
            length = mention['len']
            term = mention['str']

            drug_label_text.append([drug_name, section, start, length, term])
drug_label_text = pd.DataFrame(drug_label_text, columns=['drug_name', 'section', 'start', 'len', 'term'])
drug_label_text.sort_values(by = 'drug_name').head()

  0%|          | 0/101 [00:00<?, ?it/s]

100%|██████████| 101/101 [00:00<00:00, 135.35it/s]


,drug_name,section,start,len,term
11196,ADCETRIS,S1,17377,7,dyspnea
11052,ADCETRIS,S1,516,20,Tumor Lysis Syndrome
11053,ADCETRIS,S1,595,18,Increased Toxicity
11054,ADCETRIS,S1,715,18,Increased Toxicity
11055,ADCETRIS,S1,849,14,Hepatotoxicity


In [4]:
data = pd.read_csv('data/train_drug_label_text.csv').query('section_name == "adverse reactions"')
data.sort_values(by = 'drug_name').head()

,drug_name,section_name,section_text
212,ADCETRIS,adverse reactions,6 ADVERSE REACTIONS\n\n The following ser...
59,ADREVIEW,adverse reactions,6 ADVERSE REACTIONS\n\n EXCERPT: Seriou...
157,AFINITOR,adverse reactions,6 ADVERSE REACTIONS\n\n The following...
237,AMPYRA,adverse reactions,6 ADVERSE REACTIONS\n\n Because clinical ...
176,AMYVID,adverse reactions,6 ADVERSE REACTIONS\n\n EXCERPT: The mo...


In [5]:
data.section_text[212][595:(595+18)]

'Increased Toxicity'

In [112]:
count = 0
overlap_data = []
context_size = 100
for row in drug_label_text.itertuples():
    drug = row.drug_name
    sub_data = data[data.drug_name == drug].section_text.values[0]
    adverse_event = row.term
    if len(row.start.split(',')) <= 1:
        before = re.sub(r'[^A-Za-z ]+', '', 
        sub_data[int(row.start)-context_size:int(row.start)]).split()
        after = re.sub(r'[^A-Za-z ]+', '',
        sub_data[int(row.start)+int(row.len):int(row.start)+int(row.len)+context_size]).split()
        term = sub_data[int(row.start):int(row.start)+int(row.len)]
        num_overlap = randint(0, 10)
        num_extra_words = min([5, len(before), len(after)])
        for i in range(num_overlap):
            option = randint(0, 5)
            if option == 0: # start - X: end + Y
                start_diff = randint(1, num_extra_words) * -1
                end_diff = randint(0, num_extra_words)
                overlap_data.append([drug, adverse_event,
                                     ' '.join(before[start_diff:-1] + [term] + after[0:end_diff]), 1])
            elif option == 1: # start + X: end + Y
                if len(term.split()) > 1:
                    start_diff = randint(0, len(term.split())-1)
                    sub_term = term.split()[start_diff:]
                    end_diff = randint(1, num_extra_words)
                    overlap_data.append([drug, adverse_event,
                                          ' '.join(sub_term + after[0:end_diff]), 1])
                else:
                    overlap_data.append([drug, adverse_event,
                                        ' '.join([term] + after[0:end_diff]), 1])
            elif option == 2: # start - X: end - Y
                if len(term.split()) > 1:
                    end_diff = randint(1, len(term.split())-1)
                    sub_term = term.split()[:end_diff]
                    start_diff = randint(1, num_extra_words)
                    overlap_data.append([drug, adverse_event,
                                          ' '.join(before[-start_diff:] + sub_term), 1])
                else:
                    overlap_data.append([drug, adverse_event,
                                          ' '.join([term] + after[0:end_diff]), 1])
            elif option in [3,4,5]: # no overlap
                # either context after or context from before
                before_or_after = randint(0, 1)
                if before_or_after == 0:
                    try:
                        start = randint(1, (len(before))-1)
                        end = randint(start, len(before))
                        overlap_data.append([drug, adverse_event,
                                            ' '.join(before[start:end]), 0])
                    except:
                        overlap_data.append([drug, adverse_event,
                                            ' '.join(before), 0])
                else:
                    try:
                        start = randint(0, len(after))
                        end = randint(start, len(after))
                        overlap_data.append([drug, adverse_event,
                                            ' '.join(after[start:end]), 0])
                    except:
                        overlap_data.append([drug, adverse_event,
                                            ' '.join(after), 0])

6
10
10
12
12
12
12
12
12
13
12
12
12
13
13
13
13
13
13
12
12
12
12
12
12
12
12
12
12
12
12
12
13
13
13
2
2


ValueError: empty range for randrange() (1, 1, 0)